<a href="https://colab.research.google.com/github/silverstar0727/NLP_study/blob/main/%EC%8B%A4%EC%8A%B5_%EC%9E%90%EB%A3%8C%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 필요한 라이브러리 임포트
import tensorflow as tf
import numpy as np

from konlpy.tag import Twitter
import pandas as pd
import enum
import os
import re
import json
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

from preprocessing import *

ModuleNotFoundError: ignored

In [ ]:
def plot_graph(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_' + string], '')
  plt.xlabel('Epochs')
  plt.ylabel(string)
  plt.legend([string, 'val_' + string])
  plt.show()

98 99 100
bcd


## mask 만들기

In [ ]:
# sequence를 받아서 패딩
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32) # seq중 0을 기준으로 bool로 바꾼후 0으로 casting

  return seq[:, tf.mewaxis, tf.newaxis, :] # (batch_size, 1, 1, seq_len)

In [ ]:
def create_look_ahead_mask(size): # mask의 사이즈를 parameter로 받음
  mask = 1 - tf.linalg.band_part(tf.ones(size, size), -1, 0) # 하삼각행렬로 변환

  return mask

In [ ]:
def create_masks(inp, tar):
  enc_padding_mask = create_padding_mask(inp) # 인코더 패딩 마스크
  dec_padding_mask = create_padding_mask(inp) # 디코더 패딩 마스크

  # 디코더에 대한 마스크는 하삼각행렬을...
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

  return enc_padding_mask, combined_mask, dec_padding_mask

## positional encoding


In [ ]:
# 위치별 각도를 얻는 함수
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * i / 2) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
# 최종 포지셔널 인코딩
def positional_encoding(position, d_model): # position과 d_model을 parameter로 받음
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # 짝수는 sine 함수를 이용한 pos encoding
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # 홀수는 cosine 함수를 이용한 pos encoding
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaixs, ...]

  return tf.cast(pos_encoding, dtype = tf.float32) # pos encoding을 실수로 casting

## Scaled dot product

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  matmul_qk = tf.matmul(q, k, transpose_b = True) # key에 대해서 matmul 이전 transposed

  # scale matmul_qk
  dk = tf.cast(tf.shape()[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add!!!!
  if mask is not None:
    scaled_attention_logits += (mask * -1e9) # 큰 음수를 곱해줘서 mask화 시키도록함....

  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) # softmax 적용

  output = tf.matmul(attention_weights, v)  # matmul 적용

  return output, attention_weights

## Multihead Attention

## Feed forward network

# Encoder layer
(MHA, FFN, norm, dropout)

## Decoder layer
(MHA, FFN, norm, dropout)

## Encoder
(Embedding, ENC_layer)

## Decoder
(Embedding, DEC_layer)

## Transformer
(Encoder, Decoder, final_layer) + inference

## loss function

## model compile

## early stopping